In [1]:
import os, sys

import sgpp, sgml, dproc, sgutil
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print(sys.version)

from sklearn.pipeline import make_pipeline

3.12.9 (main, Mar 15 2025, 13:36:28) [GCC 13.3.0]


In [2]:
elected_comb = [
    # 2-interaction
    ['Episode_Length_minutes', 'Host_Popularity_percentage'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage'],
    ['Episode_Length_minutes', 'Number_of_Ads'],
    ['Episode_Title', 'Host_Popularity_percentage'],
    ['Episode_Title', 'Guest_Popularity_percentage'],
    ['Episode_Title', 'Number_of_Ads'],    
    ['Host_Popularity_percentage', 'Guest_Popularity_percentage'],
    ['Host_Popularity_percentage', 'Number_of_Ads'],
    ['Host_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Podcast_Name'],
    ['Episode_Title', 'Podcast_Name'],  
    ['Guest_Popularity_percentage', 'Podcast_Name'],
    
    # 3-interaction
    ['Episode_Length_minutes', 'Episode_Title', 'Host_Popularity_percentage'],
    ['Episode_Length_minutes', 'Episode_Title', 'Guest_Popularity_percentage'],
    ['Episode_Length_minutes', 'Episode_Title', 'Number_of_Ads'],
    ['Episode_Length_minutes', 'Episode_Title', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Episode_Title', 'Publication_Day'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Publication_Day'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Publication_Time'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage', 'Publication_Day'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage', 'Publication_Time'],
    ['Episode_Length_minutes', 'Number_of_Ads', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Number_of_Ads', 'Publication_Day'],
    ['Episode_Length_minutes', 'Episode_Sentiment', 'Publication_Time'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Guest_Popularity_percentage'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Publication_Day'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Publication_Time'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Genre'],
    ['Episode_Title', 'Guest_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Title', 'Guest_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Title', 'Guest_Popularity_percentage', 'Publication_Day'],
    ['Episode_Title', 'Guest_Popularity_percentage', 'Publication_Time'],
    ['Episode_Title', 'Guest_Popularity_percentage', 'Genre'],
    ['Episode_Title', 'Number_of_Ads', 'Episode_Sentiment'],
    ['Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads'],
    ['Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Episode_Sentiment'],
    ['Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Publication_Day'],
    ['Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Publication_Time'],
    ['Host_Popularity_percentage', 'Number_of_Ads', 'Publication_Day'],

    ['Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment'],
    ['Guest_Popularity_percentage', 'Number_of_Ads', 'Genre'],    
    
    # 4-interaction
    ['Episode_Length_minutes', 'Episode_Title', 'Host_Popularity_percentage', 'Guest_Popularity_percentage'],
    ['Episode_Length_minutes', 'Episode_Title', 'Host_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Length_minutes', 'Episode_Title', 'Host_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Episode_Title', 'Host_Popularity_percentage', 'Publication_Day'],
    ['Episode_Length_minutes', 'Episode_Title', 'Host_Popularity_percentage', 'Publication_Time'],
    ['Episode_Length_minutes', 'Episode_Title', 'Host_Popularity_percentage', 'Genre'],
    ['Episode_Length_minutes', 'Episode_Title', 'Guest_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Length_minutes', 'Episode_Title', 'Guest_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Episode_Title', 'Guest_Popularity_percentage', 'Publication_Day'],
    ['Episode_Length_minutes', 'Episode_Title', 'Guest_Popularity_percentage', 'Publication_Time'],
    ['Episode_Length_minutes', 'Episode_Title', 'Number_of_Ads', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Episode_Title', 'Number_of_Ads', 'Publication_Day'],
    ['Episode_Length_minutes', 'Episode_Title', 'Number_of_Ads', 'Publication_Time'],
    ['Episode_Length_minutes', 'Episode_Title', 'Publication_Day', 'Publication_Time'],
    ['Episode_Length_minutes', 'Episode_Title', 'Publication_Day', 'Genre'],    
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Publication_Day'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Publication_Time'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Number_of_Ads', 'Publication_Day'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time'],
    ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Publication_Day', 'Genre'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Publication_Day'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Publication_Time'],
    ['Episode_Length_minutes', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Genre'],
    ['Episode_Length_minutes', 'Episode_Title', 'Publication_Time', 'Podcast_Name'],
    
    ['Episode_Title', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Episode_Sentiment'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Number_of_Ads', 'Publication_Day'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Number_of_Ads', 'Publication_Time'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Episode_Sentiment', 'Publication_Day'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Episode_Sentiment', 'Publication_Time'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Episode_Sentiment', 'Genre'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Publication_Time', 'Genre'],
    ['Episode_Title', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment'],
    ['Episode_Title', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Genre'],
    ['Episode_Title', 'Host_Popularity_percentage', 'Episode_Sentiment', 'Podcast_Name'],
    ['Host_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment', 'Podcast_Name'],
    ['Host_Popularity_percentage', 'Number_of_Ads', 'Publication_Day', 'Podcast_Name'],
    ['Host_Popularity_percentage', 'Number_of_Ads', 'Publication_Time', 'Podcast_Name'],
    
]

X_tgt = [','.join(i) for i in elected_comb]

In [9]:
sc = sgutil.SGCache('img', 'result', 'model')
p = make_pipeline(
    sgpp.PolarsProcessor(),
    sgpp.ExprProcessor({
        'ELm_num': (pl.col('Episode_Length_minutes').fill_null(pl.col('Episode_Length_minutes').mean()).clip(5, 120) - 5) / 115,
        'GP': (pl.col('Guest_Popularity_percentage').fill_null(pl.col('Guest_Popularity_percentage').clip(0,100).mean()).clip(0, 100) / 100),
        'HP': pl.col('Host_Popularity_percentage').clip(0, 100) / 100,
        'NAd': (pl.when(pl.col('Number_of_Ads') > 4).then(0).otherwise(pl.col('Number_of_Ads'))).fill_null(0.0) /3 ,
        'Number_of_Ads': (pl.when(pl.col('Number_of_Ads') > 4).then(0).otherwise(pl.col('Number_of_Ads'))).fill_null(0).cast(pl.Int8),
        'ELm_na': pl.col('Episode_Length_minutes').is_null(),
        'GPp_na': pl.col('Guest_Popularity_percentage').is_null(),
        'ELm_sqrt': ((pl.col('Episode_Length_minutes').fill_null(pl.col('Episode_Length_minutes').mean()).clip(5, 120) - 5) / 115)  ** 0.5
    }),
    sgpp.PandasConverter(index_col = 'id'),
    sgpp.ApplyWrapper(
        sgpp.CatCombiner2(elected_comb), pd.Series(elected_comb).explode().unique().tolist()
    )
)
p.fit(['data/train.csv'])
df_train = p.transform(['data/train.csv'])
df_test = p.transform(['data/test.csv'])

In [10]:
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import KFold, cross_validate, cross_val_score, ShuffleSplit
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector
kf = KFold(5, shuffle = True, random_state = 123)
ss = ShuffleSplit(1, train_size = 0.8, random_state = 123)
target = 'Listening_Time_minutes'
X_all = [i for i in df_train.columns if i != target]

def get_validation_splitter(validation_fraction):
    return lambda x: train_test_split(x, test_size = validation_fraction)

config = {
    'predict_func': lambda m, df, X: pd.Series(m.predict(df[X]), index = df.index),
    'score_func': lambda df, prds: root_mean_squared_error(df[target], prds),
    'validation_splitter': get_validation_splitter,
    'progress_callback': sgml.ProgressCallBack(), 
    'return_train_scores': True,
    'y': target,
}

lr_adapter = sgml.SklearnAdapter(LinearRegression)
lgb_adapter = sgml.LGBMAdapter(lgb.LGBMRegressor, progress = 50)
xgb_adapter = sgml.XGBAdapter(xgb.XGBRegressor, progress = 50)

# Linear Regression 1

In [12]:
hparams = {
    'X_tgt': X_tgt,
    'X_num': ['GP', 'HP', 'NAd', 'ELm_sqrt', 'ELm_num', 'ELm_na']
}

result = sc.cv_result('LR_1', df_train, kf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result], rerun = 1)
np.mean(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

# LGB 1

In [13]:
hparams = {
    'model_params': {'num_leaves': 1024, 'n_estimators': 1500, 'learning_rate': 0.01, 'colsample_bytree': 0.7},
    'X_tgt': X_tgt, 'X_num': ['GP', 'HP', 'NAd', 'ELm_num', 'ELm_na'],
    'validation_fraction': 0.1, 'metric': 'rmse'
    #'X_cat': ['Episode_Length_minutes'], 'cat': {'handle_unknown': 'use_encoded_value', 'unknown_value': 1000000, 'encoded_missing_value': 1000000, 'min_frequency': 2}
}

result = sgml.cv(df_train, ss, hparams, config, lgb_adapter, rerun=0)
np.mean(result['valid_scores'])

Fold:   0%|          | 0/1 [00:00<?, ?it/s]

Round:   0%|          | 0/1500 [00:00<?, ?it/s]

/home/sun9sun9/python312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/sun9sun9/python312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


12.015577160698056

In [14]:
result['model_result'][0]['valid_result'].idxmin()

metric  set     
l2      training    1499
        valid_1     1464
dtype: int64

In [20]:
result['model_result'][0]['feature_importance'].sort_values(ascending = False).iloc[:50]

pt__HP                                                                                     49728
tgt__Host_Popularity_percentage,Episode_Sentiment                                          46090
tgt__Episode_Title,Podcast_Name                                                            45112
tgt__Host_Popularity_percentage,Number_of_Ads,Publication_Day                              43784
tgt__Host_Popularity_percentage,Number_of_Ads                                              43033
tgt__Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment                           41735
tgt__Episode_Length_minutes,Episode_Sentiment,Publication_Time                             39769
pt__GP                                                                                     38903
tgt__Episode_Title,Number_of_Ads,Episode_Sentiment                                         38663
tgt__Episode_Length_minutes,Podcast_Name                                                   38143
tgt__Episode_Title,Number_of_A

In [28]:
hparams = {
    'X_tgt': result['model_result'][0]['feature_importance'].sort_values(ascending = False).iloc[:50].index.to_series().str.split('__', expand = True).pipe(
                lambda x: x.loc[x[0] == 'tgt']
            )[1].tolist(),
    'X_num': ['GP', 'HP', 'NAd', 'ELm_sqrt', 'ELm_num', 'ELm_na']
}

result = sc.cv_result('LR_1', df_train, ss, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result], rerun = 1)
np.mean(result['valid_scores'])

Fold:   0%|          | 0/1 [00:00<?, ?it/s]

12.902246414939077

In [29]:
X_tgt2 = ['Host_Popularity_percentage,Episode_Sentiment',
 'Episode_Title,Podcast_Name',
 'Host_Popularity_percentage,Number_of_Ads,Publication_Day',
 'Host_Popularity_percentage,Number_of_Ads',
 'Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment',
 'Episode_Length_minutes,Episode_Sentiment,Publication_Time',
 'Episode_Title,Number_of_Ads,Episode_Sentiment',
 'Episode_Length_minutes,Podcast_Name',
 'Episode_Title,Number_of_Ads',
 'Guest_Popularity_percentage,Podcast_Name',
 'Guest_Popularity_percentage,Number_of_Ads,Genre',
 'Episode_Length_minutes,Number_of_Ads,Publication_Day',
 'Episode_Length_minutes,Number_of_Ads,Episode_Sentiment',
 'Episode_Length_minutes,Number_of_Ads',
 'Episode_Title,Host_Popularity_percentage',
 'Episode_Length_minutes,Guest_Popularity_percentage',
 'Episode_Length_minutes,Host_Popularity_percentage',
 'Episode_Length_minutes,Episode_Title,Episode_Sentiment',
 'Episode_Length_minutes,Episode_Title,Publication_Time,Podcast_Name',
 'Episode_Title,Guest_Popularity_percentage',
 'Episode_Length_minutes,Episode_Title,Publication_Day,Publication_Time',
 'Episode_Length_minutes,Episode_Title,Publication_Day',
 'Episode_Length_minutes,Episode_Title,Publication_Day,Genre',
 'Episode_Length_minutes,Episode_Title,Number_of_Ads,Publication_Time',
 'Episode_Length_minutes,Episode_Title,Number_of_Ads,Publication_Day',
 'Episode_Length_minutes,Host_Popularity_percentage,Publication_Time',
 'Episode_Length_minutes,Host_Popularity_percentage,Episode_Sentiment',
 'Episode_Length_minutes,Guest_Popularity_percentage,Publication_Time',
 'Episode_Length_minutes,Episode_Title,Number_of_Ads',
 'Episode_Length_minutes,Host_Popularity_percentage,Number_of_Ads',
 'Episode_Length_minutes,Episode_Title,Number_of_Ads,Episode_Sentiment',
 'Episode_Length_minutes,Guest_Popularity_percentage,Number_of_Ads',
 'Episode_Length_minutes,Guest_Popularity_percentage,Publication_Day',
 'Episode_Length_minutes,Host_Popularity_percentage,Publication_Day',
 'Episode_Title,Guest_Popularity_percentage,Publication_Time',
 'Episode_Title,Guest_Popularity_percentage,Episode_Sentiment',
 'Episode_Title,Guest_Popularity_percentage,Publication_Day',
 'Episode_Length_minutes,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment',
 'Episode_Length_minutes,Host_Popularity_percentage,Number_of_Ads,Episode_Sentiment',
 'Episode_Title,Guest_Popularity_percentage,Number_of_Ads',
 'Episode_Length_minutes,Guest_Popularity_percentage,Number_of_Ads,Publication_Time',
 'Episode_Title,Host_Popularity_percentage,Episode_Sentiment',
 'Episode_Title,Guest_Popularity_percentage,Genre',
 'Host_Popularity_percentage,Guest_Popularity_percentage',
 'Episode_Title,Host_Popularity_percentage,Number_of_Ads',
 'Episode_Title,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment',
 'Episode_Title,Host_Popularity_percentage,Publication_Time']

In [30]:
hparams = {
    'model_params': {'num_leaves': 1024, 'n_estimators': 1500, 'learning_rate': 0.01, 'colsample_bytree': 0.7},
    'X_tgt': X_tgt2, 'X_num': ['GP', 'HP', 'NAd', 'ELm_num', 'ELm_na'],
    'validation_fraction': 0.1, 'metric': 'rmse'
    #'X_cat': ['Episode_Length_minutes'], 'cat': {'handle_unknown': 'use_encoded_value', 'unknown_value': 1000000, 'encoded_missing_value': 1000000, 'min_frequency': 2}
}

result = sgml.cv(df_train, ss, hparams, config, lgb_adapter, rerun=0)
np.mean(result['valid_scores'])

Fold:   0%|          | 0/1 [00:00<?, ?it/s]

Round:   0%|          | 0/1500 [00:00<?, ?it/s]

/home/sun9sun9/python312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/sun9sun9/python312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


12.106815321332654

In [32]:
result['model_result'][0]['feature_importance'].sort_values(ascending = False).iloc[50:100]

pt__NAd       6437
pt__ELm_na    1407
dtype: int32

In [34]:
from sklearn.model_selection import train_test_split
df_train_tgt1, df_test_tgt1 = train_test_split(
    df_train, train_size = 0.75, random_state =123, shuffle=True
)

In [36]:
from sklearn.preprocessing import TargetEncoder

tgt = TargetEncoder(cv = 4, random_state = 123)
tgt.fit(df_train_tgt1[['Host_Popularity_percentage,Episode_Sentiment']], df_train_tgt1[target])

TargetEncoder(cv=4, random_state=123)

In [38]:
root_mean_squared_error(
    df_test_tgt1[target], tgt.transform(df_test_tgt1[['Host_Popularity_percentage,Episode_Sentiment']])[:, 0]
)

27.110308008418933

In [40]:
tgt.fit(df_train_tgt1[['Episode_Length_minutes,Episode_Sentiment,Publication_Time']], df_train_tgt1[target])
root_mean_squared_error(
    df_test_tgt1[target], tgt.transform(df_test_tgt1[['Episode_Length_minutes,Episode_Sentiment,Publication_Time']])[:, 0]
)

15.045093824885647

In [41]:
tgt.fit(df_train_tgt1[['Episode_Length_minutes']], df_train_tgt1[target])
root_mean_squared_error(
    df_test_tgt1[target], tgt.transform(df_test_tgt1[['Episode_Length_minutes']])[:, 0]
)

13.402868075299962

In [46]:
hparams = {
    'X_tgt': ['Episode_Length_minutes,Episode_Sentiment,Publication_Time', 'Episode_Length_minutes', 'Host_Popularity_percentage,Episode_Sentiment'],
    'X_num': ['GP', 'HP', 'NAd', 'ELm_sqrt', 'ELm_num', 'ELm_na']
}

result = sc.cv_result('LR_1', df_train, ss, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result], rerun = 1)
np.mean(result['valid_scores'])

Fold:   0%|          | 0/1 [00:00<?, ?it/s]

13.137181914208908

In [47]:
hparams = {
    'model_params': {'num_leaves': 1024, 'n_estimators': 1500, 'learning_rate': 0.01, 'colsample_bytree': 0.7},
    'X_tgt': ['Episode_Length_minutes,Episode_Sentiment,Publication_Time', 'Episode_Length_minutes', 'Host_Popularity_percentage,Episode_Sentiment'],'X_num': ['GP', 'HP', 'NAd', 'ELm_num', 'ELm_na'],
    'validation_fraction': 0.1, 'metric': 'rmse'
    #'X_cat': ['Episode_Length_minutes'], 'cat': {'handle_unknown': 'use_encoded_value', 'unknown_value': 1000000, 'encoded_missing_value': 1000000, 'min_frequency': 2}
}

result = sgml.cv(df_train, ss, hparams, config, lgb_adapter, rerun=0)
np.mean(result['valid_scores'])

Fold:   0%|          | 0/1 [00:00<?, ?it/s]

Round:   0%|          | 0/1500 [00:00<?, ?it/s]

/home/sun9sun9/python312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/sun9sun9/python312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


12.853681399016606